In [ ]:
import requests
import pandas as pd
import re
import time
import json
from tqdm import tqdm
from openai import OpenAI
import os
from concurrent.futures import ThreadPoolExecutor
import google.generativeai as genai

In [ ]:
OPENAI_API_KEY = "my_key1"
ANTHROPIC_API_KEY = "my_key2"
GEMINI_API_KEY = "my_key3"

In [3]:
# ------------------------------------
# 🔧 Configuration
# ------------------------------------
API_KEY = OPENAI_API_KEY  # 🔐 Replace this with your actual key
MODEL = "gpt-4.1-mini"
INPUT_FILE = "../Openalex/openalex_non_cs_2024.csv"
OUTPUT_FILE_ALL = "stage 1/All Methods.csv"
OUTPUT_FILE_FILTERED = "stage 1/Methods Filtered.csv"
SLEEP_SECONDS = 1

client = OpenAI(api_key=API_KEY)

# ------------------------------------
# 📚 Step 1: Get Method Definition
# ------------------------------------
def get_method_definition(model=MODEL) -> str:
    prompt = """
In the context of AI, Machine Learning, and Deep Learning, define what a 'method' is. Your definition should focus on how methods are used to solve research problems or objectives, such as specific algorithms, techniques, or approaches. Avoid examples. Return only a single, clear sentence definition.
"""
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=150
    )
    return response.choices[0].message.content.strip()

# ------------------------------------
# ✍️ Step 2: Create Prompt for Abstract Extraction
# ------------------------------------
def make_method_extraction_prompt(abstract: str, method_definition: str) -> str:
    return f"""
Definition: {method_definition}

Based on the above definition of a method, your task is to extract all Artificial Intelligence, Machine Learning, or Deep Learning methods/methodologies mentioned in the follorwing research abstract that were used to address the research problem or objective.

Here is the abstract:
---
{abstract}
---

Wrap each method in <method>...</method> tags and separate each method with a comma. If there are no valid Artificial Intelligence, Machine Learning, or Deep Learning methods then just respond with: No methods found.
"""

# ------------------------------------
# 🤖 Step 3: Extract Methods from Abstract
# ------------------------------------
def extract_methods_from_abstract(abstract: str, method_definition: str, model: str = MODEL) -> str:
    prompt = make_method_extraction_prompt(abstract, method_definition)
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=800
    )
    return response.choices[0].message.content.strip()

# ------------------------------------
# 🔁 Step 4: Apply Extraction on All Abstracts
# ------------------------------------
def process_abstracts(input_path: str, method_definition: str) -> pd.DataFrame:
    df = pd.read_csv(input_path)
    results = []

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        try:
            title = row.get("title", "")
            abstract = row.get("abstract", "")
            year = row.get("publication_year", "")
            year = row.get("publication_year", "")
            openalex_id = row.get("openalex_id", "")
            domain = row.get("domain_source", "")

            print(f"🔍 [{idx}] Processing abstract: {title[:60]}...")
            method_tags = extract_methods_from_abstract(abstract, method_definition)

            results.append({
                "year": year,
                "openalex_id": openalex_id,
                "domain": domain,
                "title": title,
                "abstract": abstract,
                "extracted methods": method_tags
            })

            time.sleep(SLEEP_SECONDS)

        except Exception as e:
            print(f"❌ [{idx}] Error: {e}")
            continue

    return pd.DataFrame(results)

# ------------------------------------
# 💾 Step 5: Save Outputs with Directory Fix
# ------------------------------------
def save_method_extractions(df: pd.DataFrame, output_all: str, output_filtered: str):
    # Ensure output directories exist
    os.makedirs(os.path.dirname(output_all), exist_ok=True)
    os.makedirs(os.path.dirname(output_filtered), exist_ok=True)

    # Save all results
    df.to_csv(output_all, index=False)
    print(f"✅ All results saved to: {output_all}")

    # Save only rows with <method> tags
    df_filtered = df[df["extracted methods"].str.contains("<method>", na=False)]
    df_filtered.to_csv(output_filtered, index=False)
    print(f"✅ Filtered results saved to: {output_filtered}")
    print(f"📊 Filtered rows: {len(df_filtered)} / {len(df)}")

# ------------------------------------
# 🎬 Main
# ------------------------------------
if __name__ == "__main__":
    print("📚 Getting AI/ML/DL method definition from GPT...")
    method_definition = get_method_definition()
    print(f"\n✅ Definition used:\n{method_definition}\n")

    with open("method_definition_used.json", "w") as f:
        json.dump({"Method": method_definition}, f, indent=2)

    df_results = process_abstracts(INPUT_FILE, method_definition)
    save_method_extractions(df_results, OUTPUT_FILE_ALL, OUTPUT_FILE_FILTERED)


📚 Getting AI/ML/DL method definition from GPT...

✅ Definition used:
A method in AI, Machine Learning, and Deep Learning is a systematic algorithmic or procedural approach designed to address specific research problems or objectives by enabling models to learn patterns, make predictions, or derive insights from data.



  0%|                                                                                         | 0/1606 [00:00<?, ?it/s]

🔍 [0] Processing abstract: A Comprehensive Survey of Continual Learning: Theory, Method...


  0%|                                                                                 | 1/1606 [00:01<47:47,  1.79s/it]

🔍 [1] Processing abstract: Global prediction of extreme floods in ungauged watersheds...


...

🔍 [1604] Processing abstract: Pseudo-spectral angle mapping for automated pixel-level anal...


100%|█████████████████████████████████████████████████████████████████████████████▉| 1605/1606 [48:55<00:01,  1.51s/it]

🔍 [1605] Processing abstract: Perspectives and challenges in developing and implementing i...


100%|██████████████████████████████████████████████████████████████████████████████| 1606/1606 [48:57<00:00,  1.83s/it]

✅ All results saved to: stage 1/All Methods.csv
✅ Filtered results saved to: stage 1/Methods Filtered.csv
📊 Filtered rows: 1065 / 1606


In [4]:
# ----------------------------
# 🔐 API KEYS (set yours here)
# ----------------------------

# Initialize OpenAI
openai_client = OpenAI(api_key=OPENAI_API_KEY)

# Configure Gemini
genai.configure(api_key=GEMINI_API_KEY)

# Claude setup
CLAUDE_URL = "https://api.anthropic.com/v1/messages"
CLAUDE_HEADERS = {
    "x-api-key": ANTHROPIC_API_KEY,
    "anthropic-version": "2023-06-01",
    "content-type": "application/json"
}

# ----------------------------
# 📥 Load Extracted Methods
# ----------------------------
df = pd.read_csv("stage 1/Methods Filtered.csv")

all_methods = []
for entry in df["extracted methods"]:
    if isinstance(entry, str) and "No methods found" not in entry:
        methods = re.findall(r"<method>(.*?)</method>", entry)
        all_methods.extend(method.strip() for method in methods)

unique_methods = sorted(set(all_methods))
print(f"📊 Unique candidate methods: {len(unique_methods)}")

# ----------------------------
# 🧠 Prompt Template
# ----------------------------
def make_prompt(method_name):
    return f"""Determine if the following phrase represents an actual real Artificial Intelligence, Machine Learning, or Deep Learning method.

Phrase: "{method_name}"

Respond only with "Yes" or "No"."""

# ----------------------------
# 🔍 Model 1: GPT-4o-mini
# ----------------------------
def check_with_gpt(method):
    try:
        response = openai_client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[{"role": "user", "content": make_prompt(method)}],
            temperature=0.10
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {str(e)}"

# ----------------------------
# 🔍 Model 2: Claude 3.5 Sonnet
# ----------------------------
def check_with_claude(method):
    try:
        payload = {
            "model": "claude-3-7-sonnet-20250219",
            "max_tokens": 1024,
            "messages": [{"role": "user", "content": make_prompt(method)}]
        }
        response = requests.post(CLAUDE_URL, headers=CLAUDE_HEADERS, json=payload)
        response.raise_for_status()
        return response.json()["content"][0]["text"].strip()
    except Exception as e:
        return f"Error: {str(e)}"

# ----------------------------
# 🔍 Model 3: Gemini 1.5 Pro
# ----------------------------
def check_with_gemini(method):
    try:
        model = genai.GenerativeModel("gemini-1.5-pro")
        response = model.generate_content(make_prompt(method))
        return response.text.strip()
    except Exception as e:
        return f"Error: {str(e)}"

# ----------------------------
# 🧪 Validate with All Three (Parallel)
# ----------------------------
results = []

for method in tqdm(unique_methods, desc="🔁 Validating Methods (Parallel)"):
    with ThreadPoolExecutor() as executor:
        f_gpt = executor.submit(check_with_gpt, method)
        f_claude = executor.submit(check_with_claude, method)
        f_gemini = executor.submit(check_with_gemini, method)

        gpt_result = f_gpt.result()
        claude_result = f_claude.result()
        gemini_result = f_gemini.result()

    print(f"\nMethod: {method}")
    print(f"  GPT    → {gpt_result}")
    print(f"  Claude → {claude_result}")
    print(f"  Gemini → {gemini_result}")

    results.append({
        "method": method,
        "GPT": gpt_result,
        "Claude": claude_result,
        "Gemini": gemini_result
    })

df_results = pd.DataFrame(results)
df_results.to_csv("stage 2/tri_model_method_validation.csv", index=False)
print("\n✅ Saved: tri_model_method_validation.csv")

📊 Unique candidate methods: 1955


🔁 Validating Methods (Parallel):   0%|                                             | 1/1955 [00:03<1:41:44,  3.12s/it]


Method: 10-fold cross-validation
  GPT    → No
  Claude → Yes
  Gemini → Yes


🔁 Validating Methods (Parallel):   0%|                                             | 2/1955 [00:05<1:31:43,  2.82s/it]


Method: 1D convolution neural network (CNN)
  GPT    → Yes
  Claude → Yes
  Gemini → Yes


...


Method: zero-shot sequential action prediction
  GPT    → Yes
  Claude → Yes
  Gemini → Yes


🔁 Validating Methods (Parallel): 100%|█████████████████████████████████████████▉| 1954/1955 [1:51:14<00:02,  2.84s/it]


Method: zero‐shot learning
  GPT    → Yes
  Claude → Yes
  Gemini → Yes


🔁 Validating Methods (Parallel): 100%|██████████████████████████████████████████| 1955/1955 [1:51:19<00:00,  3.42s/it]


Method: ℓ1-regularized sparse modeling
  GPT    → Yes
  Claude → Yes
  Gemini → Yes

✅ Saved: tri_model_method_validation.csv


In [5]:
df_results = pd.read_csv("stage 2/tri_model_method_validation.csv")

# ----------------------------
# ✅ Consensus Filtering: All 3 Must Say "Yes"
# ----------------------------
df_results["GPT"] = df_results["GPT"].str.strip().str.lower()
df_results["Claude"] = df_results["Claude"].str.strip().str.lower()
df_results["Gemini"] = df_results["Gemini"].str.strip().str.lower()

agree_methods = df_results[
    (df_results["GPT"] == "yes") &
    (df_results["Claude"] == "yes") &
    (df_results["Gemini"] == "yes")
]["method"].tolist()

print(f"✅ Methods agreed by all 3: {len(agree_methods)}")

# ----------------------------
# ✂️ Filter Original Method CSV
# ----------------------------
def keep_only_agreed(entry):
    if not isinstance(entry, str):
        return entry
    methods = re.findall(r"<method>(.*?)</method>", entry)
    filtered = [m for m in methods if m.strip() in agree_methods]
    return ''.join(f"<method>{m}</method>" for m in filtered) if filtered else "No methods remaining"

print(df.columns.tolist())
df["verified extracted methods"] = df["extracted methods"].apply(keep_only_agreed)
df.to_csv("stage 3/methods_filtered_tri_model_all_rows.csv", index=False)
df_valid = df[df["verified extracted methods"] != "No methods remaining"]
df_valid.to_csv("stage 3/methods_filtered_tri_model_rows_only.csv", index=False)

print(f"✅ Saved cleaned data")
print(f"📊 Valid rows retained: {len(df_valid)} / {len(df)}")

✅ Methods agreed by all 3: 1001
['year', 'openalex_id', 'domain', 'title', 'abstract', 'extracted methods']
✅ Saved cleaned data
📊 Valid rows retained: 792 / 1065


In [6]:
# Load your CSV
df = pd.read_csv("stage 3/methods_filtered_tri_model_rows_only.csv")

# Function to extract methods from tags into a list
def extract_methods_as_list(tagged_text):
    if not isinstance(tagged_text, str):
        return []
    return re.findall(r"<method>(.*?)</method>", tagged_text)

# Create new list column
df["verified_methods_list"] = df["verified extracted methods"].apply(extract_methods_as_list)

# Drop the old tagged methods column
df.drop(columns=["extracted methods"], inplace=True)
df.drop(columns=["verified extracted methods"], inplace=True)

# Save the cleaned DataFrame
df.to_csv("stage 3/methods_with_verified_methods_list.csv", index=False)

print("✅ Cleaned file saved to 'methods_with_verified_methods_list.csv' with only list format methods.")


✅ Cleaned file saved to 'methods_with_verified_methods_list.csv' with only list format methods.


In [9]:
distribution = pd.read_csv("stage 3/methods_with_verified_methods_list.csv")
print("📊 Paper count by primary domain:")
print(distribution["domain"].value_counts())

📊 Paper count by primary domain:
Medicine           212
Biology            173
Engineering        155
Psychology         132
Social Sciences    120
Name: domain, dtype: int64
